https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html

https://nikolasent.github.io/computervision/opencv/calibration/2024/12/20/Practical-OpenCV-Refinement-Techniques.html

In [4]:
import glob as glob
import os

import numpy as np

import calibrationlib

# This has to be done once theoritically, if the cameras are securely fixed together.

session_path = (r"./data/calibration/classical_system")

# Calibration paths for left ('Camera1') and right ('Camera2') chessboard images
calib_paths = [
    session_path + "/Left",
    session_path + "/Right",
]

image_type = "jpg"

# Chessboard pattern size (width, height)
chessboard_pattern = (13, 8)
# Chessboard pattern cell size (here in mm) : used for scaling calibration parameters
chessboard_cell_size = 40

calibration = calibrationlib.CameraCalibration(chessboard_pattern, chessboard_cell_size)

count = 0

object_points_cam1 = []
image_points_cam1 = []
object_points_cam2 = []
image_points_cam2 = []

success_images_cam1 = []
success_images_cam2 = []

name = session_path + "/errors.log"
log = open(name, "w")

In [5]:
# Computation of the calibration parameters for both left and right cameras, then for the stereo pair

# First camera
log_line = "Calibration - First camera\r"
log.write(log_line)

ret1 = False
if not (os.path.exists(calib_paths[0] + "/image_points.npy")):
    (
        ret1,
        reprojection_error_cam1,
        success_images_cam1,
        roi_cam1,
        camera_matrix_cam1,
        dist_coefs_cam1,
        r_vecs_cam1,
        t_vecs_cam1,
        object_points_cam1,
        image_points_cam1,
        image_size_cam1,
    ) = calibration.single_calibration(
        calib_paths[0],
        image_type,
        log_file=log,
    )

    tmp = np.array(object_points_cam1)
    name = calib_paths[0] + "/object_points"
    np.save(name, tmp)
    tmp = np.array(image_points_cam1)
    name = calib_paths[0] + "/image_points"
    np.save(name, tmp)
    tmp = np.array(success_images_cam1)
    name = calib_paths[0] + "/success_images"
    np.save(name, tmp)

    tmp = np.array(image_size_cam1)
    name = calib_paths[0] + "/image_size"
    np.save(name, tmp)

    tmp = np.array(camera_matrix_cam1)
    name = calib_paths[0] + "/camera_matrix"
    np.save(name, tmp)
    tmp = np.array(dist_coefs_cam1)
    name = calib_paths[0] + "/dist_coefs"
    np.save(name, tmp)
    tmp = np.array(r_vecs_cam1)
    name = calib_paths[0] + "/r_vecs"
    np.save(name, tmp)
    tmp = np.array(t_vecs_cam1)
    name = calib_paths[0] + "/t_vecs"
    np.save(name, tmp)
    tmp = np.array(reprojection_error_cam1)
    name = calib_paths[0] + "/reprojection_error"
    np.save(name, tmp)
else:
    camera_matrix_cam1 = np.load(calib_paths[0] + "/camera_matrix.npy")
    dist_coefs_cam1 = np.load(calib_paths[0] + "/dist_coefs.npy")
    image_size_cam1 = np.load(calib_paths[0] + "/image_size.npy")
    object_points_cam1 = np.load(calib_paths[0] + "/object_points.npy")
    object_points_cam1 = list(object_points_cam1)
    image_points_cam1 = np.load(calib_paths[0] + "/image_points.npy")
    image_points_cam1 = list(image_points_cam1)
    success_images_cam1 = np.load(calib_paths[0] + "/success_images.npy")
    success_images_cam1 = list(success_images_cam1)

    ret1 = True

In [6]:
dist_coefs_cam1

array([[ 0.01741913],
       [-0.00377566],
       [ 0.00063558],
       [-0.00152854],
       [-0.01996182]])

In [7]:
# Right camera
log_line = "Calibration - Second camera\r"
log.write(log_line)

ret2 = False
if not (os.path.exists(calib_paths[1] + "/image_points.npy")):
    (
        ret2,
        reprojection_error_cam2,
        success_images_cam2,
        roi_cam2,
        camera_matrix_cam2,
        dist_coefs_cam2,
        r_vecs_cam2,
        t_vecs_cam2,
        object_points_cam2,
        image_points_cam2,
        img_size_cam2,
    ) = calibration.single_calibration(
        calib_paths[1],
        image_type,
        log_file=log,
    )

    tmp = np.array(object_points_cam2)
    name = calib_paths[1] + "/object_points"
    np.save(name, tmp)
    tmp = np.array(image_points_cam2)
    name = calib_paths[1] + "/image_points"
    np.save(name, tmp)
    tmp = np.array(success_images_cam2)
    name = calib_paths[1] + "/success_images"
    np.save(name, tmp)

    tmp = np.array(image_size_cam1)
    name = calib_paths[1] + "/image_size"
    np.save(name, tmp)
    
    tmp = np.array(camera_matrix_cam2)
    name = calib_paths[1] + "/camera_matrix"
    np.save(name, tmp)
    tmp = np.array(dist_coefs_cam2)
    name = calib_paths[1] + "/dist_coefs"
    np.save(name, tmp)
    tmp = np.array(r_vecs_cam2)
    name = calib_paths[1] + "/r_vecs"
    np.save(name, tmp)
    tmp = np.array(t_vecs_cam2)
    name = calib_paths[1] + "/t_vecs"
    np.save(name, tmp)
    tmp = np.array(reprojection_error_cam2)
    name = calib_paths[1] + "/reprojection_error"
    np.save(name, tmp)
else:
    camera_matrix_cam2 = np.load(calib_paths[1] + "/camera_matrix.npy")
    dist_coefs_cam2 = np.load(calib_paths[1] + "/dist_coefs.npy")
    image_size_cam1 = np.load(calib_paths[0] + "/image_size.npy")
    object_points_cam2 = np.load(calib_paths[1] + "/object_points.npy")
    object_points_cam2 = list(object_points_cam2)
    image_points_cam2 = np.load(calib_paths[1] + "/image_points.npy")
    image_points_cam2 = list(image_points_cam2)
    success_images_cam2 = np.load(calib_paths[1] + "/success_images.npy")
    success_images_cam2 = list(success_images_cam2)
    ret2 = True

In [8]:
dist_coefs_cam2

array([[ 0.02194274],
       [-0.0166134 ],
       [ 0.00044922],
       [-0.00137957],
       [ 0.00328596]])

In [9]:
# Calibration of the stereo pair
log_line = "Stereo calibration\r"
log.write(log_line)

if (ret1 is True) and (ret2 is True):

    # We keep only image pairs on which were successfully detected the chessboard corners
    # for both cameras, and store relative objects and image points for further stereo
    # calibration
    shortest_list_cam1 = success_images_cam1
    longest_list_cam1 = success_images_cam2
    if len(success_images_cam1) > len(success_images_cam2):
        shortest_list_cam1 = success_images_cam2
        longest_list_cam1 = success_images_cam1

    index_points_cam1 = []
    index_points_cam2 = []
    for el in shortest_list_cam1:
        if el in longest_list_cam1:
            index_points_cam1.append(success_images_cam1.index(el))
            index_points_cam2.append(success_images_cam2.index(el))

    log_line = f"Common images : {(np.array(success_images_cam1))[index_points_cam1]}\r"
    log.write(log_line)
    log_line = f"Images indexes for camera 1 : {index_points_cam1}\r"
    log.write(log_line)
    log_line = f"Images indexes for camera 2 : {index_points_cam2}\r"
    log.write(log_line)

    # Resulting list of common points for the rtwo cameras
    filtered_object_points_cam1 = np.array(object_points_cam1)
    filtered_object_points_cam1 = filtered_object_points_cam1[index_points_cam1]
    filtered_object_points_cam1 = list(filtered_object_points_cam1)
    
    filtered_image_points_cam1 = np.array(image_points_cam1)
    filtered_image_points_cam1 = filtered_image_points_cam1[index_points_cam1]
    filtered_image_points_cam1 = list(filtered_image_points_cam1)
    
    filtered_image_points_cam2 = np.array(image_points_cam2)
    filtered_image_points_cam2 = filtered_image_points_cam2[index_points_cam2]
    filtered_image_points_cam2 = list(filtered_image_points_cam2)

    # We compute now stereo calibration : camera_matrices and distorsion coefficients for each
    # camera may be slightly modified (depending on the flags parameters : in our case, the output 
    # values are strictly the same as the input ones), and we get the rotation matrix R and 
    # translation matrix T that bring points in the left camera coordinate system to points in the 
    # right camera coordinate system (you may read the doc : documentation/OpenCV - stereoCalibrate) 
    (
        ret,
        camera_matrix_cam1_stereo,
        dist_coefs_cam1_stereo,
        camera_matrix_cam2_stereo,
        dist_coefs_cam2_stereo,
        R,
        T,
    ) = calibration.stereo_calibration(
        filtered_object_points_cam1,
        filtered_image_points_cam1,
        filtered_image_points_cam2,
        camera_matrix_cam1,
        dist_coefs_cam1,
        camera_matrix_cam2,
        dist_coefs_cam2,
        image_size_cam1,
        log_file=log,
    )

    # We save the useful parameters, from which further rectification of images will 
    # be possible for matching. The rectification maps will be loaded once at the 
    # beginning of the matching code, and applied to each images pair.
    tmp = np.array(camera_matrix_cam1_stereo)
    name = calib_paths[0] + "/camera_matrix_from_stereo"
    np.save(name, tmp)
    tmp = np.array(dist_coefs_cam1_stereo)
    name = calib_paths[0] + "/dist_coefs_from_stereo"
    np.save(name, tmp)
    tmp = np.array(camera_matrix_cam2_stereo)
    name = calib_paths[1] + "/camera_matrix_from_stereo"
    np.save(name, tmp)
    tmp = np.array(dist_coefs_cam2_stereo)
    name = calib_paths[1] + "/dist_coefs_from_stereo"
    np.save(name, tmp)
    tmp = np.array(R)
    name = session_path + "/R"
    np.save(name, tmp)
    tmp = np.array(T)
    name = session_path + "/T"
    np.save(name, tmp)
    tmp = np.array(ret)
    name = session_path + "/reprojection_error"
    np.save(name, tmp)
else:
    print("No way to perform stereo calibration")

print("End of calibration process")

End of calibration process
